# Equipment_move_workflow

In [ ]:
## FROM : Ancienne table, INNER JOIN : Nouvelle base 

/*delimiter //
CREATE TRIGGER `move_equipment` BEFORE INSERT ON `IT_Equipment_records`
 FOR EACH ROW BEGIN
    DECLARE _Reperage_ID , _Nom_de_la_salle, _INSERTED varchar(255);
    
        SELECT Reperage_ID, Nom_de_la_salle, INSERTED 
        INTO _Reperage_ID, _Nom_de_la_salle, _INSERTED
        FROM `IT_Equipment_records`
        WHERE ID_Systeme_Asset = NEW.ID_Systeme_Asset
        ORDER BY STR_TO_DATE((inserted),'%y-%m-%d') DESC
        LIMIT 1 ;
    
        IF(
          (NEW.reperage_id != _Reperage_ID OR 
          NEW.Nom_de_la_salle != _Nom_de_la_salle) AND 
          ( NEW.ID_Systeme_Asset != '0' or NEW.ID_Systeme_Asset != ' ' )
        )
        THEN
        INSERT INTO move_records_equipments (asset_id, ID_Systeme_Asset, modele, from_salle, from_reperage_id, to_salle, to_reperage_id, from_date, to_date) 
           VALUES (NEW.asset_id, NEW.ID_Systeme_Asset, NEW.modele, _Nom_de_la_salle, _reperage_id, new.Nom_de_la_salle, new.reperage_id, _INSERTED, new.inserted);
        END IF;
           
END
*/

In [ ]:
drop view if exists IT_Equipment_sorted_by_date_asc ;
create view `IT_Equipment_sorted_by_date_asc` 
as
select ASSET_ID, ID_Systeme_Asset, Modele, Nom_de_la_salle, Reperage_ID, SOURCES, INSERTED from `IT_Equipment_records` 
ORDER BY STR_TO_DATE((inserted),'%y-%m-%d') ASC ;

drop view if exists IT_Equipment_sorted_by_date_desc ;
create view `IT_Equipment_sorted_by_date_desc` 
as
select ASSET_ID, ID_Systeme_Asset, Modele, Nom_de_la_salle, Reperage_ID, SOURCES, INSERTED from `IT_Equipment_records` 
ORDER BY STR_TO_DATE((inserted),'%y-%m-%d') DESC ;

In [ ]:
drop view if exists IT_Equipment_first_occurence_per_id_all ;
CREATE view IT_Equipment_first_occurence_per_id_all
as 
select min(inserted) first_records,it.* from `IT_Equipment_sorted_by_date_asc` it
GROUP BY ID_Systeme_Asset
order by INSERTED desc ;

In [ ]:
drop view if exists IT_Equipment_first_occurence_per_id ;
CREATE view  IT_Equipment_first_occurence_per_id
as
select it.* from `IT_Equipment_records` it
INNER JOIN `IT_Equipment_first_occurence_per_id_all` it2 
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset
AND it.inserted = it2.first_records ;

Tous les mouvements par rapport à la position de la première date d'insertion

In [ ]:
drop view if exists IT_Equipment_all_move;
create view IT_Equipment_all_move
as
select it.id_systeme_asset id_systeme_asset_records,it.nom_de_la_salle salle_records, it.reperage_id reperage_id_records, it.inserted inserted_records, it2.*
from `IT_Equipment_records` it
INNER JOIN `IT_Equipment_first_occurence_per_id` it2
ON 
(( it.reperage_id != it2.reperage_id
OR it.nom_de_la_salle != it2.nom_de_la_salle )
AND it.id_systeme_asset = it2.id_systeme_asset )

      
where it2.id_systeme_asset is not null

order by it.ID_Systeme_Asset asc , STR_TO_DATE((it2.inserted),'%y-%m-%d') DESC;# MySQL returned an empty result set (i.e. zero rows).

In [ ]:
drop view if exists IT_Equipment_all_move_with_last_date ;
create VIEW IT_Equipment_all_move_with_last_date
as
select  MAX(inserted_records) last_records, it.*  from `IT_Equipment_all_move` it
GROUP BY id_systeme_asset_records ;

In [ ]:
drop view if exists IT_Equipment_last_move;
CREATE VIEW IT_Equipment_last_move
as
SELECT it.* from `IT_Equipment_all_move` it 
INNER JOIN `IT_Equipment_all_move_with_last_date` it2 
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset
AND it.inserted_records = it2.last_records
;

synthèse des mouvements par asset_id

SELECT ID_Systeme_Asset, Nom, count(concat(nom_de_la_salle,Reperage_ID)) row_number, count(DISTINCT(concat(nom_de_la_salle,Reperage_ID))) - 1 number_of_move FROM `IT_Equipment_records`

GROUP BY ID_Systeme_Asset
HAVING row_number > 2 and number_of_move >= 1
order by count(Reperage_ID) asc ;

select  MAX(inserted_records) last_records ,it.* from `IT_Equipment_all_move` it
GROUP BY id_systeme_asset_records ;

In [ ]:
drop table if exists `IT_Equipment_unique_records` ;
create table `IT_Equipment_unique_records`
as 
select * from `IT_Equipment_records`
GROUP BY ID_Systeme_Asset ;

In [ ]:
ALTER TABLE IT_Equipment_unique_records
ADD COLUMN first_occurence varchar(255),
ADD COLUMN last_move varchar(255),
ADD COLUMN last_move_date varchar(255),
ADD COLUMN clean_date varchar(255) ;

In [ ]:
INSERT `IT_Equipment_records` values ("Equipements IT" ,NULL,NULL,"FRC2VPN9" ,"RN000081" ,"Cisco" ,"ASA 5516-X" ,"P4" ,"F30" ,26 ,NULL,NULL,"Serveurs" ,NULL,"FRC2VPN9" ,"JMX2036Y1LF" ,"0" ,"RN000081" ,"M-0000775783" ,"0" ,"Puissance nominale et facteur" ,"0.06" ,NULL,NULL,"60.0" ,"0.036000000000000004" ,"MonophasÃ©" ,"On" ,"1.0" ,"1.0" ,"1.0" ,"1.0" ,"Single Source" ,"Q142A" ,"Prise 13" ,"3.0" ,NULL,NULL,"TDI_P4A" ,"Autres" ,NULL,NULL,NULL,"Q142A (16 A)" ,"Phase 3" ,NULL,NULL,NULL,NULL,NULL,NULL,"InstallÃ©" ,"2018-04-19T00:00:00.000Z" ,NULL,NULL,NULL,"AAF30:Slot 26",NULL,NULL,"Avant" ,"Pas de rotation" ,"MontÃ©(s) en baie" ,"FRC2VPN9" ,NULL,"27F" ,NULL,NULL,"0.983" ,"0.43700000000000006" ,"0.287" ,"3.0" ,NULL,"0 ","1238131-88444" ,"400171-124799" ,NULL,"IT_Equipment.11.05.22.csv" ,"21-04-22")

In [ ]:
INSERT `IT_Equipment_records` values ("Equipements IT" ,NULL,NULL,"FRC2VPN9" ,"RN000081" ,"Cisco" ,"ASA 5516-X" ,"P4" ,"F30" ,26 ,NULL,NULL,"Serveurs" ,NULL,"FRC2VPN9" ,"JMX2036Y1LF" ,"0" ,"RN000081" ,"M-0000775783" ,"0" ,"Puissance nominale et facteur" ,"0.06" ,NULL,NULL,"60.0" ,"0.036000000000000004" ,"MonophasÃ©" ,"On" ,"1.0" ,"1.0" ,"1.0" ,"1.0" ,"Single Source" ,"Q142A" ,"Prise 13" ,"3.0" ,NULL,NULL,"TDI_P4A" ,"Autres" ,NULL,NULL,NULL,"Q142A (16 A)" ,"Phase 3" ,NULL,NULL,NULL,NULL,NULL,NULL,"InstallÃ©" ,"2018-04-19T00:00:00.000Z" ,NULL,NULL,NULL,"AAF30:Slot 26",NULL,NULL,"Avant" ,"Pas de rotation" ,"MontÃ©(s) en baie" ,"FRC2VPN9" ,NULL,"27F" ,NULL,NULL,"0.983" ,"0.43700000000000006" ,"0.287" ,"3.0" ,NULL,"0 ","1238131-88444" ,"400171-124799" ,NULL,"IT_Equipment.11.05.22.csv" ,"22-09-22")

# First occurence

In [ ]:
UPDATE `IT_Equipment_unique_records` it
INNER JOIN `IT_Equipment_first_occurence_per_id` it2
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset

set first_occurence = it2.inserted

# Last move

In [ ]:
UPDATE `IT_Equipment_unique_records` it
INNER JOIN `IT_Equipment_last_move` it2
ON it.ID_Systeme_Asset = it2.ID_Systeme_Asset

set last_move_date = it2.inserted_records,
    last_move = it2.reperage_id_records